In [387]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [388]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [389]:
spectrum_train_scaled = spectrum_train.multiply(10**12)
spectrum_test_scaled = spectrum_test.multiply(10**12)

In [390]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [391]:
input_data = df_to_tensor(temp_train)
output_data = df_to_tensor(spectrum_train_scaled)

In [392]:
input_data

tensor([[350.6424, 357.4240, 338.0000,  ..., 267.9975, 252.9999, 238.0000],
        [352.1875, 360.3629, 342.0451,  ..., 269.5426, 252.9999, 236.4549],
        [350.6424, 357.4240, 338.0000,  ..., 267.9975, 252.9999, 238.0000],
        ...,
        [597.3622, 585.6775, 574.4026,  ..., 493.7609, 505.0429, 515.4084],
        [595.8171, 582.7385, 570.3576,  ..., 492.2158, 505.0429, 516.9536],
        [601.9975, 594.4943, 586.5379,  ..., 498.3961, 505.0429, 510.7732]])

In [393]:
output_data

tensor([[ 0.1119,  0.0821,  0.0643,  ...,  0.3437,  0.3360,  0.3290],
        [ 0.1240,  0.0904,  0.0701,  ...,  0.3269,  0.3197,  0.3131],
        [ 0.1119,  0.0821,  0.0643,  ...,  0.3437,  0.3360,  0.3290],
        ...,
        [10.8134, 10.6301, 10.5865,  ..., 21.8142, 20.7956, 19.8604],
        [10.6221, 10.5147, 10.5277,  ..., 22.0530, 21.0220, 20.0754],
        [11.4288, 11.0081, 10.7881,  ..., 21.1053, 20.1236, 19.2221]])

In [394]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [395]:
input_size = 11
output_size = 66
model = Net(input_size=input_size, output_size=output_size)

In [396]:
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.00001)  # Adam optimizer

In [397]:
num_epochs = 15
batch_size = 1
num_batches = len(input_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        inputs = input_data[start:end]
        targets = output_data[start:end]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

Epoch 1/15, Loss: 26.873066
Epoch 2/15, Loss: 20.303852
Epoch 3/15, Loss: 18.222498


KeyboardInterrupt: 

In [ ]:
predictions = model(df_to_tensor(temp_train))

In [ ]:
pred = pd.DataFrame(predictions.detach().numpy(), columns=spectrum_test.columns)

In [ ]:
actual = spectrum_test_scaled

In [ ]:
100 + actual.subtract(pred).divide(pred).multiply(100).mean().mean()

80.31123236613274